# Estimación de Modelo Probit

### Conjunto de datos: participación laboral femenina

* En este ejercicio trabajamos en el conjunto de datos de Mroz sobre la participación laboral femenina con 8 variables.

* Los datos cubren una muestra de 753 mujeres blancas casadas de entre 30 y 60 años en 1975. 

* La fuente original de estos datos es Mroz, T.A. (1987). “The sensitivity of an empirical model of married women’s hours of work to economic and statistical assumptions.” Econometrica 55, 765-799.

* La descripción de las variables se puede encontrar a continuación:

lfp: Labor-force participation of the married white woman (Categorical: 0/1)

k5: Number of children younger than 6 years old	(Entero positivo)

k618: Number of children aged 6-18 (Entero positivo)

age: Age in years (Entero positivo)

wc: Wife's college attendance (Categorical: 0/1)

hc: Husband's college attendance (Categorical: 0/1)

lwg: Log expected wage rate for women in the labor force (Numerical)

inc: Family income without the wife's income (Numerical)

# 1. Importación de bibliotecas y datos

In [1]:
# Dependecies:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit

#
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación de datos
data = pd.read_csv('Mroz.csv')
data.head()

,lfp,k5,k618,age,wc,hc,lwg,inc
0,yes,1,0,32,no,no,1.210165,10.910001
1,yes,0,2,30,no,no,0.328504,19.500000
2,yes,1,3,35,no,no,1.514128,12.039999
3,yes,0,3,34,no,no,0.092115,6.800000
4,yes,1,2,31,yes,no,1.524280,20.100000


In [3]:
# Limpieza de datos:
#
data["lfp_d"] = 0
data.loc[data.lfp == "yes", "lfp_d"] = 1

#
data["wc_d"] = 0
data.loc[data.wc == "yes", "wc_d"] = 1

#
data["hc_d"] = 0
data.loc[data.hc == "yes", "hc_d"] = 1

#
data

,lfp,k5,k618,age,wc,hc,lwg,inc,lfp_d,wc_d,hc_d
0,yes,1,0,32,no,no,1.210165,10.910001,1,0,0
1,yes,0,2,30,no,no,0.328504,19.500000,1,0,0
2,yes,1,3,35,no,no,1.514128,12.039999,1,0,0
3,yes,0,3,34,no,no,0.092115,6.800000,1,0,0
4,yes,1,2,31,yes,no,1.524280,20.100000,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
748,no,0,2,40,yes,yes,1.082864,28.200001,0,1,1
749,no,2,3,31,no,no,1.158040,10.000000,0,0,0
750,no,0,0,43,no,no,0.888140,9.952000,0,0,0
751,no,0,0,60,no,no,1.224974,24.983999,0,0,0


In [4]:
# Estadísticas descriptivas:

data.describe()

,k5,k618,age,lwg,inc,lfp_d,wc_d,hc_d
count,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000,753.000000
mean,0.237716,1.353254,42.537849,1.097115,20.128965,0.568393,0.281541,0.391766
std,0.523959,1.319874,8.072574,0.587556,11.634799,0.495630,0.450049,0.488469
min,0.000000,0.000000,30.000000,-2.054124,-0.029000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,36.000000,0.818086,13.025000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,43.000000,1.068403,17.700001,1.000000,0.000000,0.000000
75%,0.000000,2.000000,49.000000,1.399717,24.466000,1.000000,1.000000,1.000000
max,3.000000,8.000000,60.000000,3.218876,96.000000,1.000000,1.000000,1.000000


# 2. Estimación del modelo Probit

In [5]:
# Definimos variables dependiente e independientes:

Y = data['lfp_d']
X = data[['k5', 'k618', 'age', 'wc_d', 'hc_d', 'lwg', 'inc']]
X = sm.add_constant(X)

In [6]:
# Estimación:

model = Probit(Y, X.astype(float))
probit_model = model.fit()
print(probit_model.summary())

Optimization terminated successfully.
         Current function value: 0.601189
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                  lfp_d   No. Observations:                  753
Model:                         Probit   Df Residuals:                      745
Method:                           MLE   Df Model:                            7
Date:                Wed, 12 May 2021   Pseudo R-squ.:                  0.1208
Time:                        08:46:00   Log-Likelihood:                -452.69
converged:                       True   LL-Null:                       -514.87
Covariance Type:            nonrobust   LLR p-value:                 9.471e-24
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9184      0.381      5.040      0.000       1.172       2.664
k5            -0.8747      0.

# 3. Efectos marginales

Algunas caracterísricas de los datos:

In [7]:
# Tablas de frecuencias por categoria para hc: Husband's college attendance (Categorical: 0/1)

pd.crosstab(data["lfp"], data["hc"], margins = True)

hc,no,yes,All
lfp,,,
no,207,118,325
yes,251,177,428
All,458,295,753


\begin{eqnarray*}
    EMg_j & = & P(y = 1 | \mathbf{x}_i, x_j = 1) - P(y = 1 | \mathbf{x}_i, x_j = 0) \\
    & = & G(\mathbf{x}_i \boldsymbol{\beta} | x_j = 1) - G(\mathbf{x}_i \boldsymbol{\beta} | x_j = 0) \\
    & = & G(\beta_1 + x_{2} \beta_2 + \ldots + \beta_j + \ldots + x_{K} \beta_K) \\
    &  & - G(\beta_1 + x_{2} \beta_2 + \ldots + 0 + \ldots + x_{K} \beta_K)
\end{eqnarray*}

In [8]:
# hc: Husband's college attendance = 0
hc_data0 = np.column_stack((
                            1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]),
                            np.mean(data["wc_d"]), 0, np.mean(data["lwg"]), np.mean(data["inc"])
                           ))

hc_data0

array([[ 1.        ,  0.2377158 ,  1.35325365, 42.53784861,  0.2815405 ,
         0.        ,  1.09711483, 20.12896541]])

In [9]:
# hc: Husband's college attendance = 1
hc_data1 = np.column_stack((
                            1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]),
                            np.mean(data["wc_d"]), 1, np.mean(data["lwg"]), np.mean(data["inc"])
                           ))

hc_data1

array([[ 1.        ,  0.2377158 ,  1.35325365, 42.53784861,  0.2815405 ,
         1.        ,  1.09711483, 20.12896541]])

In [10]:
# Estimación para el primero: D = 0
print(probit_model.predict(hc_data0))

[0.56938181]


In [11]:
# Estimación para el segundo: D = 1
print(probit_model.predict(hc_data1))

[0.59171968]


In [12]:
# Tablas de frecuencias por categoria para wc: Wife's college attendance (Categorical: 0/1)

pd.crosstab(data["lfp"], data["wc"], margins = True)

wc,no,yes,All
lfp,,,
no,257,68,325
yes,284,144,428
All,541,212,753


In [13]:
# wc: Wife's college attendance = 0
wc_data0 = np.column_stack((1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]), 
                            0, np.mean(data["hc_d"]), np.mean(data["lwg"]), np.mean(data["inc"])))

wc_data0

array([[ 1.        ,  0.2377158 ,  1.35325365, 42.53784861,  0.        ,
         0.39176627,  1.09711483, 20.12896541]])

In [14]:
# wc: Wife's college attendance = 0
wc_data1 = np.column_stack((1, np.mean(data["k5"]), np.mean(data["k618"]), np.mean(data["age"]), 
                            1, np.mean(data["hc_d"]), np.mean(data["lwg"]), np.mean(data["inc"])))

wc_data1

array([[ 1.        ,  0.2377158 ,  1.35325365, 42.53784861,  1.        ,
         0.39176627,  1.09711483, 20.12896541]])

In [15]:
print(probit_model.predict(wc_data0)), print(probit_model.predict(wc_data1))

[0.52380974]
[0.70816505]


(None, None)

In [16]:
# Tablas de frecuencias por categoria para k5: Number of children younger than 6 years old (Entero positivo)

pd.crosstab(data["lfp"], data["k5"], margins = True)

k5,0,1,2,3,All
lfp,,,,,
no,231,72,19,3,325
yes,375,46,7,0,428
All,606,118,26,3,753


\begin{eqnarray*}
    EMg_j & = & P(y = 1 | \mathbf{x}_i, x_j = 1) - P(y = 1 | \mathbf{x}_i, x_j = 0) \\
    & = & G(\mathbf{x}_i \boldsymbol{\beta} | x_j = C + 1) - G(\mathbf{x}_i \boldsymbol{\beta} | x_j = C) \\
\end{eqnarray*}

In [17]:
k5_data = np.column_stack(( np.repeat(1,4), (0,1,2,3),
                           np.repeat(np.mean(data["k618"]), 4), np.repeat(np.mean(data["age"]),4),
                           np.repeat(np.mean(data["wc_d"]),4), np.repeat(np.mean(data["hc_d"]),4),
                           np.repeat(np.mean(data["lwg"]),4), np.repeat(np.mean(data["inc"]),4) ))

k5_data

array([[ 1.        ,  0.        ,  1.35325365, 42.53784861,  0.2815405 ,
         0.39176627,  1.09711483, 20.12896541],
       [ 1.        ,  1.        ,  1.35325365, 42.53784861,  0.2815405 ,
         0.39176627,  1.09711483, 20.12896541],
       [ 1.        ,  2.        ,  1.35325365, 42.53784861,  0.2815405 ,
         0.39176627,  1.09711483, 20.12896541],
       [ 1.        ,  3.        ,  1.35325365, 42.53784861,  0.2815405 ,
         0.39176627,  1.09711483, 20.12896541]])

In [18]:
print(probit_model.predict(k5_data))

[0.65730924 0.31932735 0.08942703 0.01324326]


Sin importar el modelo que estemos ocupando, la forma de interpretar el modelo es mediante el efecto marginal, cuando $x_j$ sea una variable continua:
\begin{equation*}
    EMg_j = \frac{\partial}{\partial x_j} P(y = 1 | \mathbf{x}_i) = \frac{\partial}{\partial x_j} G(\mathbf{x}_i \boldsymbol{\beta}) = g(\mathbf{x}_i \boldsymbol{\beta}) \beta_j
\end{equation*}

In [19]:
#
mfx = probit_model.get_margeff()

print(mfx.summary())

       Probit Marginal Effects       
Dep. Variable:                  lfp_d
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
k5            -0.2997      0.034     -8.726      0.000      -0.367      -0.232
k618          -0.0132      0.014     -0.955      0.340      -0.040       0.014
age           -0.0130      0.002     -5.219      0.000      -0.018      -0.008
wc_d           0.1673      0.045      3.696      0.000       0.079       0.256
hc_d           0.0196      0.042      0.461      0.645      -0.064       0.103
lwg            0.1253      0.029      4.311      0.000       0.068       0.182
inc           -0.0070      0.002     -4.451      0.000      -0.010      -0.004
